In [ ]:
import atoti as tt
import psycopg2 

session = tt.Session(name="dashboardPuskesmas", user_content_storage="./DashA1")

conn = psycopg2.connect(
        host = "********",
        port = 1111,
        database = "PG_SKRIPSI",
        user = "********",
        password ="********"          
    )   
    
query=("select fact_kunjungan.factkunjungankey, dim_waktu.tanggal,dim_waktu.notahun, dim_waktu.namaBulan, dim_waktu.quarter, dim_waktu.semester, dim_dusun.nama_dusun, dim_dusun.nama_desa,dim_penyakit.nama_penyakit, dim_poliklinik.nama_poliklinik, dim_golongan_umur.golongan_umur,fact_kunjungan.jenis_kelamin, fact_kunjungan.jumlah_biaya from fact_kunjungan left join dim_waktu on dim_waktu.waktukey = fact_kunjungan.waktukey left join dim_penyakit on dim_penyakit.penyakitkey = fact_kunjungan.penyakitkey LEFT JOIN dim_dusun on dim_dusun.dusunKey=fact_kunjungan.dusunkey LEFT JOIN dim_poliklinik on dim_poliklinik.poliklinikKey=fact_kunjungan.poliklinikkey LEFT JOIN dim_golongan_umur on dim_golongan_umur.golumurKey=fact_kunjungan.golumurkey")

cursor = conn.cursor()
cursor.execute(query)

import pandas as pd
data = pd.DataFrame(cursor.fetchall(), columns=["factkunjungankey", "tanggal", "notahun","namabulan","quarter","semester","nama_dusun","nama_desa","nama_penyakit","nama_poliklinik","golongan_umur","jenis_kelamin","jumlah_biaya"])

table = session.read_pandas(data, keys=["factkunjungankey"], table_name="fact_kunjungan")
cube = session.create_cube(table)

m = cube.measures
m

l = cube.levels
l

h = cube.hierarchies
h

h["lokasi"] = [
    table["nama_desa"],
    table["nama_dusun"],
]

h["waktu"] = [
    table["notahun"],
    table["semester"],
    table["quarter"],
    table["namabulan"],
    table["tanggal"],
]

del h["notahun"]
del h["semester"]
del h["quarter"]
del h["namabulan"]
del h["tanggal"]
del h["nama_dusun"]
del h["nama_desa"]

h = cube.hierarchies
h

l = cube.levels
l

session.link()
